In [50]:
import pandas as pd
import re
import requests
from io import StringIO

In [51]:
df = pd.read_csv('enron_mails_subsample.csv')
df

,message_id,subject,body,category
0,<23778978.1075863635437.JavaMail.evans@thyme>,Re: Security Assessment Report,I appreciate you pressing this issue. You shou...,"Company Business, Strategy, etc.(elaborate in ..."
1,<31218520.1075847578460.JavaMail.evans@thyme>,Re: eBiz article on Bush energy plan,Looks good to me. I'm glad you had the chance ...,"Personal but in professional context(e.g., it ..."
2,<24162646.1075858884386.JavaMail.evans@thyme>,UC-CSU-Enron press release,I know we got a lot of grief on the Hill when ...,"Company Business, Strategy, etc.(elaborate in ..."
3,<22094025.1075842958662.JavaMail.evans@thyme>,NaN,When we have described the problems and soluti...,"Company Business, Strategy, etc.(elaborate in ..."
4,<18617596.1075846170274.JavaMail.evans@thyme>,Re: MidAmerican Draft Documents,"I realize you have pushed these guys already, ...",Document editing/checking(collaboration)
...,...,...,...,...
1395,<26310133.1075847587043.JavaMail.evans@thyme>,REVISED: Meeting with Antoinette Malveaux re: ...,meeting file ---------------------- Forwarded ...,"Logistic Arrangements(meeting scheduling, tech..."
1396,<16219455.1075847592739.JavaMail.evans@thyme>,<<Concur Expense Document>> - General Expenses,---------------------- Forwarded by Steven J K...,"Logistic Arrangements(meeting scheduling, tech..."
1397,<4992356.1075849344192.JavaMail.evans@thyme>,Energy Issues,"Please see the following articles: Sac Bee, Fr...","Company Business, Strategy, etc.(elaborate in ..."
1398,<9054756.1075851974341.JavaMail.evans@thyme>,Ken Lay Mention At NRSC Reception Last Night,Last night I attended the National Republican ...,"Company Business, Strategy, etc.(elaborate in ..."


In [44]:
'''
labels=[]
with open('1.txt', 'r') as file:
    for line in file:
        li=line.strip()
        print(li)
        li=re.findall('(^.*?):',li)
        labels.append(li)
label=[i for sl in labels for i in sl]
label=label[:-1]
# label.append('Body')
label
'''

"\nlabels=[]\nwith open('1.txt', 'r') as file:\n    for line in file:\n        li=line.strip()\n        print(li)\n        li=re.findall('(^.*?):',li)\n        labels.append(li)\nlabel=[i for sl in labels for i in sl]\nlabel=label[:-1]\n# label.append('Body')\nlabel\n"

In [45]:
df['Message-ID']=df['RawMailContent'].str.extract('Message-ID'+r':\s(.*)\n')
df['Subject']=df['RawMailContent'].str.extract('Subject'+r':\s(.*)\n')
df['Body']=df['RawMailContent'].str.extract('X-FileName'+r':\s.*\n([\s\S]*)$')
df.drop('RawMailContent',axis=1,inplace=True)
df['Body']=df['Body'].replace({'\n':' ','\t':' '},regex=True)

df['Body']=df['Body'].replace({'"':'',"'":""},regex=True)
df['Subject']=df['Subject'].replace({'"':'',"'":""},regex=True)

df.loc[df['Body'].str.contains('-----Original Message-----'),'Body']
df.loc[df['Body'].str.contains('-----Original Message-----'),'Body']

0         Forwarded for tracking purposes.  -----Origi...
2        Tana,  Hope you are well.  I wondered whether...
12       Debra Perlingiere Enron North America Corp. L...
13       Bob -  I believe what we sent them was a form...
17       This one is easy, Texas Gas k#T018123 was rec...
                              ...                        
4967     I thought this one was funny enough to forwar...
4972     it was really good.  howd you cut up those da...
4979     Alan, heres the open season announcement I se...
4986     I was reading through your and others recomme...
4997     Attached below is a 9 page summary of what Ch...
Name: Body, Length: 737, dtype: object

In [46]:
df.loc[df['Body'].str.contains('-----Original Message-----'),'Body']=df.loc[df['Body'].str.contains('-----Original Message-----'),'Body'].str.extract(r'([\s\S]*?)'+r'-----Original Message-----',expand=False)
# 
df

,Message-ID,Subject,Body
0,<21713028.1075855434284.JavaMail.evans@thyme>,FW: Notice of Termination - Power Purchase and...,Forwarded for tracking purposes.
1,<30445252.1075855073917.JavaMail.evans@thyme>,,Return Receipt Your document: was recei...
2,<12158479.1075847131465.JavaMail.evans@thyme>,RE: FW: Deutsche Bank ISDA Master Agreements,"Tana, Hope you are well. I wondered whether..."
3,<13684295.1075845928316.JavaMail.evans@thyme>,Re: FW: VEPCO/City of Rocky Mount,"Im travelling on Monday, so please ask my ass..."
4,<2352172.1075842967105.JavaMail.evans@thyme>,,"If theres other stuff you need, or if youd li..."
...,...,...,...
4995,<10566838.1075846391124.JavaMail.evans@thyme>,Letter to John Pope,Attached is a new draft of the letter to John...
4996,<32232039.1075847075437.JavaMail.evans@thyme>,Re: New CAs & Revised List,One comment - the Platts agreement has been c...
4997,<9769889.1075858707282.JavaMail.evans@thyme>,FW: Summary of Sections to be marked up,Attached below is a 9 page summary of what Ch...
4998,<31276235.1075846020705.JavaMail.evans@thyme>,World Hunger- Section 9.6 Additional tests - t...,9.6 Additional Factory Testing . In the eve...


In [47]:
''' for a in label:
    if not df['RawMailContent'].str.contains(a).any():
        continue
    #if df['RawMailContent'].str.extract(a+r':\s*(.*)').iloc[0,0] in label:
     #   continue
    df[a]=df['RawMailContent'].str.extract(a+r':\s(.*)\n')
    
    df['RawMailContent']=df['RawMailContent'].str.replace(a+r':\s(.*)\n','')
    df['RawMailContent']=df['RawMailContent'].str.strip('\n')
df
'''

" for a in label:\n    if not df['RawMailContent'].str.contains(a).any():\n        continue\n    #if df['RawMailContent'].str.extract(a+r':\\s*(.*)').iloc[0,0] in label:\n     #   continue\n    df[a]=df['RawMailContent'].str.extract(a+r':\\s(.*)\n')\n    \n    df['RawMailContent']=df['RawMailContent'].str.replace(a+r':\\s(.*)\n','')\n    df['RawMailContent']=df['RawMailContent'].str.strip('\n')\ndf\n"

In [48]:
# df[~(df['X-bcc']=='')]

In [49]:
df['Body']=df['Body'].str.strip()
df['Subject']=df['Subject'].str.strip()

df['Body'].iloc[0]

'Forwarded for tracking purposes.'